In [ ]:
import numpy as np
import robotic as ry

In [ ]:
C = ry.Config()
C.addFile(ry.raiPath('scenarios/pandaSingle.g'))

bot = ry.BotOp(C, True)

In [ ]:
bot.moveTo(bot.get_qHome(), 1.)
bot.wait(C)

while bot.getKeyPressed()!=ord('q'):
    bot.sync(C, .1)
    y, J = C.eval(ry.FS.position, ["l_gripper"], [[1,0,0]])
    bot.setCompliance(J, 1.)
    print(' direct:', J @ bot.get_tauExternal(),
          ' pseudoInv:', np.linalg.pinv(J.T, rcond=1e-3) @ bot.get_tauExternal())

bot.setCompliance([], 0.)

In [ ]:
# Stolen from the Jenga project :)
def linear_move(
    C,
    bot,
    target_frame,
    move_frame="finger_r",
    interpolation_steps=10,
    print_force=False,
):
    current_pos = C.getFrame(move_frame).getPosition()
    current_quat = C.getFrame(move_frame).getQuaternion()
    target_pos = target_frame.getPosition()
    pos_diff = np.array([tar - cur for tar, cur in zip(target_pos, current_pos)])
    step_size = pos_diff / interpolation_steps
    for step in range(1, interpolation_steps + 1):
        step_target = current_pos + step * step_size
        C.addFrame("step_target").setPosition(step_target).setQuaternion(
            current_quat
        ).setShape(ry.ST.marker, size=[0.05, 0.02])
        komo = ry.KOMO()
        komo.setConfig(C, True)
        komo.setTiming(2.0, 1, 5.0, 0)
        komo.addControlObjective([], 0, 1e-0)
        komo.addObjective([], ry.FS.accumulatedCollisions, [], ry.OT.eq)
        komo.addObjective([], ry.FS.jointLimits, [], ry.OT.ineq)
        komo.addObjective([], ry.FS.poseDiff, [move_frame, "step_target"], ry.OT.eq)
        # Solve program
        ret = (
            ry.NLP_Solver()
            .setProblem(komo.nlp())
            .setOptions(stopTolerance=1e-2, verbose=0)
            .solve()
        )
        C.view()
        q = komo.getPath()
        C.setJointState(q[0])
        bot.moveTo(q[0])
        max_force = -np.inf
        while bot.getTimeToEnd() > 0:
            bot.sync(C, 0.1)
            y, J = C.eval(ry.FS.position, [move_frame], [[0, 1, 0]])
            F = np.abs(J @ bot.get_tauExternal())
            max_force = F if F > max_force else max_force
        C.delFrame("step_target")
        if max_force > 0.8:
            print(max_force)
            print("Max force exeeded")
            return False
    if print_force:
        print(f"max:{max_force}")
    return True